In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [2]:
def wait1(s):
    if s <= 0: return
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait2(s):
    pass
wait = wait1

def call(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv

### Global functions and routines

In [3]:
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def settemp(sp):
    if sp == 0:
        call(h.settemp, 2, 37)
    else:
        call(h.settemp, 0, sp)
        
def endbatch():
    call(h.endbatch)
    
def mkpid(p, i):
    def pid():
        call(h.setconfig, "Temperature", "P_Gain_(%/C)", p)
        call(h.setconfig, "Temperature", 'I_Time_(min)', i)
    return pid
        

In [4]:
_bni = 1
def nb(pref, i=1, n=1):
    bl = call(h.getBatches)
    b = bl.names_to_batches
    s = "%s %d.%d" % (pref, i, n)
    while s in b:
        n += 1
        s = "%s %d.%d" % (pref, i, n)
    startbatch(s)

def tempoff():
    settemp(0)
    
def setbn(i):
    global _bni
    _bni = i

def o2a():
    nb("tpid", _bni)
    settemp(37)
    wait(3*hours)
    endbatch()
    
def cool(hrs=8):
    tempoff()
    wait(hrs*hours)
    
def a2a(end):
    nb("tpid", _bni)
    settemp(end)
    wait(3*hours)
    endbatch()

def verify(pid):
    print("Verifying settings")
    pid()
    o2a()
    a2a(35)
    a2a(37)
    
def verify2(pid):
    print("Verifying settings")
    pid()
    o2a()
 
def tpv():
    return call(h.gpmv)['temperature']['pv']

In [5]:
def waitcool(sp, t=3600):
    tempoff()
    while tpv() >= sp:
        print("waitcool")
        wait(t)

In [8]:
h = HelloApp('71.189.82.196:81')
p='C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\IE00005 PID Tuning\\IE00005-05 Temperature\\raw data\\'

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn, force=False):
    if isinstance(bn, str):
        _b = lambda: call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        bn = name
        _b = lambda: call(h.getreport_bydate, 'process_data', d1, d2)

    fn = p+bn+" 3L " + bt() + ".csv"
    if os.path.exists(fn) and not force:
        return fn
    b = _b()
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [9]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
    
def analyze(fn):
    c1 = cells.Find("TempPV(C)")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "1440"
    mkch(c1,"TempPV(C)")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    
    for v in "TempHeatDutyActual(%)",:
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        mkser(ch2, c, v)
    fixaxes()
    wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)
        
def analf(f):
    open_file(f)
    analyze(f)

In [ ]:
print("starting")
hours = 3600

h = HelloApp('71.189.82.196:81')
wait = wait1
combos = [
    (50, 6),
    (50, 7),
    (50, 8),
    (30, 10),
    (40, 8),
]
setbn(3)
call(h.setconfig, "System", "Max Data Log Interval (min)", 1)
for p, i in combos:
    waitcool(25)
    pid = mkpid(p, i)
    verify2(pid)
tempoff()

In [114]:
def gbl(n=3):
    return [b for b in call(h.getBatches).names_to_batches if b.startswith("tpid %d"%n)]
bl = gbl(3)
files = []
for b in bl:
    fn = dl(b)
    files.append(fn)
xl = Excel()
xl.DisplayAlerts = False
with screen_lock(xl):
    for file in files: analf(file)

In [115]:
pi = dict(("tpid 2.%d" % n, "P: %d I: %d" % (p, i)) for n, (p,i) in enumerate(combos, 1))
pi

{'tpid 2.1': 'P: 30 I: 10',
 'tpid 2.2': 'P: 50 I: 8',
 'tpid 2.3': 'P: 40 I: 9',
 'tpid 2.4': 'P: 30 I: 12',
 'tpid 2.5': 'P: 50 I: 7'}

In [116]:
wbs = list(xl.Workbooks)
nwb = xl.Workbooks.Add()
for wb in wbs:
    ws = wb.Worksheets(1)
    c = ws.ChartObjects(1).Chart
    tpid = " ".join(wb.Name.split()[:2])
    name = pi[tpid]
    c.SeriesCollection(1).Name = name
    FormatAxesScale(c, None, None, 25)
    c.Location(xlc.xlLocationAsNewSheet, tpid)
    c = wb.Charts(1)
    c.Copy(After=(nwb.Charts(nwb.Charts.Count) if nwb.Charts.Count > 0 else nwb.Worksheets(3)))

charts = tuple(nwb.Charts)
charts[0].Copy(After=charts[-1])
c2 = nwb.Charts(nwb.Charts.Count)
c2.Name = "All"
for c in charts[1:]:
    c.Activate()
    c.ChartArea.Copy()
    c2.Activate()
    c2.Paste()